In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import mne
import scipy.io
import shutil
from scipy.signal import resample, resample_poly, decimate

In [13]:
# Hyper Params

oldFS = 256
newFS = 32
length = 16*newFS # n*newfs [eikahne n hosse amder window length]
stride = length
p = 0.5  # eita use kortesi jate akta nirdisto row sum korar por jodi value  p ke extend kore tahole annotation 1 kore dibo

In [14]:
annotation_indices = list(range(0, 4608, 4))
annotation_indices

[0,
 4,
 8,
 12,
 16,
 20,
 24,
 28,
 32,
 36,
 40,
 44,
 48,
 52,
 56,
 60,
 64,
 68,
 72,
 76,
 80,
 84,
 88,
 92,
 96,
 100,
 104,
 108,
 112,
 116,
 120,
 124,
 128,
 132,
 136,
 140,
 144,
 148,
 152,
 156,
 160,
 164,
 168,
 172,
 176,
 180,
 184,
 188,
 192,
 196,
 200,
 204,
 208,
 212,
 216,
 220,
 224,
 228,
 232,
 236,
 240,
 244,
 248,
 252,
 256,
 260,
 264,
 268,
 272,
 276,
 280,
 284,
 288,
 292,
 296,
 300,
 304,
 308,
 312,
 316,
 320,
 324,
 328,
 332,
 336,
 340,
 344,
 348,
 352,
 356,
 360,
 364,
 368,
 372,
 376,
 380,
 384,
 388,
 392,
 396,
 400,
 404,
 408,
 412,
 416,
 420,
 424,
 428,
 432,
 436,
 440,
 444,
 448,
 452,
 456,
 460,
 464,
 468,
 472,
 476,
 480,
 484,
 488,
 492,
 496,
 500,
 504,
 508,
 512,
 516,
 520,
 524,
 528,
 532,
 536,
 540,
 544,
 548,
 552,
 556,
 560,
 564,
 568,
 572,
 576,
 580,
 584,
 588,
 592,
 596,
 600,
 604,
 608,
 612,
 616,
 620,
 624,
 628,
 632,
 636,
 640,
 644,
 648,
 652,
 656,
 660,
 664,
 668,
 672,
 676,
 680,
 6

In [15]:
def downsample(data, oldFS, newFS):
    data_annotation=data['annotation']
    data_clip=data.iloc[:,0:18]
    data=pd.concat([data_clip,data_annotation],axis=1)
    newNumSamples = int((data.shape[0] / oldFS) * newFS)
    newData = pd.DataFrame(resample(data[data.columns[:-1]], newNumSamples))
    annotation_indices = list(range(0, len(data), 16))
    annotation = data.annotation.loc[annotation_indices].reset_index(drop = True)
    newData['annotation'] = annotation
    return newData

In [16]:
%%time

path = 'eldfs/'
babydfs = list()
for file in sorted(os.listdir(path)):
    print(file)
    df = downsample(pd.read_csv(path + file), oldFS, newFS)
    finaldfs = list()
    for i in range(0, len(df), stride):
        annotation = 0
        try:
            if df[df.columns[-1]].iloc[i:i+length].value_counts()[1] > int(p * length):
                annotation = 1
        except:
            annotation = 0
        int_dfs = list()
        for j in range(18):
            window = df[df.columns[j]].iloc[i:i+length]
            int_dfs.append(window)
        int_df = pd.DataFrame(pd.concat(int_dfs, axis = 0, ignore_index = True)).T
        int_df['annotation'] = annotation

        finaldfs.append(int_df)
    finaldf = pd.concat(finaldfs, axis = 0)
    babydfs.append(finaldf)
babydf = pd.concat(babydfs, axis = 0).reset_index(drop = True).dropna(how = 'any')

eeg0.csv
eeg1.csv
eeg11.csv
eeg13.csv
eeg15.csv
eeg16.csv
eeg17.csv
eeg25.csv
eeg29.csv
eeg3.csv
eeg31.csv
eeg32.csv
eeg33.csv
eeg34.csv
eeg35.csv
eeg36.csv
eeg37.csv
eeg38.csv
eeg39.csv
eeg4.csv
eeg40.csv
eeg41.csv
eeg42.csv
eeg43.csv
eeg44.csv
eeg45.csv
eeg46.csv
eeg47.csv
eeg48.csv
eeg49.csv
eeg5.csv
eeg50.csv
eeg51.csv
eeg52.csv
eeg53.csv
eeg54.csv
eeg55.csv
eeg56.csv
eeg57.csv
eeg58.csv
eeg59.csv
eeg6.csv
eeg60.csv
eeg61.csv
eeg62.csv
eeg63.csv
eeg64.csv
eeg65.csv
eeg66.csv
eeg67.csv
eeg68.csv
eeg69.csv
eeg7.csv
eeg70.csv
eeg71.csv
eeg72.csv
eeg73.csv
eeg74.csv
eeg75.csv
eeg76.csv
eeg77.csv
eeg78.csv
eeg8.csv
eeg9.csv
CPU times: total: 15min 34s
Wall time: 16min 18s


In [17]:
babydf

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,annotation
0,-3.263788e-07,-1.235657e-06,-2.262840e-06,-7.633439e-08,-2.164782e-07,-9.267774e-07,-2.608645e-07,4.420569e-07,1.074870e-06,1.201442e-06,...,-2.108430e-08,-6.075327e-08,3.585114e-08,-7.522059e-08,-2.129476e-07,1.840603e-08,-4.448791e-08,-1.414767e-07,-1.127482e-07,0
1,-2.486152e-05,-2.503058e-05,-2.186363e-05,-2.480244e-05,-2.744258e-05,-3.197565e-05,-2.888203e-05,-2.740444e-05,-2.505538e-05,-2.380489e-05,...,-3.678916e-08,4.569411e-08,-1.483502e-09,-1.193529e-07,8.563138e-08,-8.082530e-08,6.143709e-08,7.245130e-08,-1.086324e-08,0
2,-2.500956e-07,3.655730e-07,1.434406e-06,1.338836e-06,1.395847e-06,1.193354e-06,4.941691e-07,-1.154113e-06,-1.678423e-06,-2.962719e-06,...,2.377878e-07,2.749303e-07,4.018399e-07,4.383263e-07,4.915250e-07,1.993675e-07,5.477542e-07,2.877784e-07,4.136224e-07,0
3,-1.232406e-05,-1.401220e-05,-1.413148e-05,-1.257527e-05,-1.504887e-05,-1.492065e-05,-1.218374e-05,-1.174109e-05,-1.214782e-05,-1.030836e-05,...,4.371654e-08,-2.403598e-08,-7.028860e-08,3.183884e-08,-7.033279e-09,-1.487082e-07,-2.021294e-08,1.308101e-07,-7.147434e-08,1
4,1.282229e-06,5.628105e-06,1.596520e-06,6.568708e-06,3.496735e-06,8.595807e-06,-1.840697e-06,4.598325e-05,8.422721e-05,6.828570e-05,...,-1.439245e-07,8.825039e-08,5.644418e-08,-7.229984e-08,-6.370705e-08,3.334085e-08,-1.182866e-07,-1.872063e-07,7.822432e-08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19427,-1.126685e-05,-1.047553e-05,-1.210227e-05,-1.163751e-05,-1.239892e-05,-1.125410e-05,-1.029162e-05,-9.223068e-06,-7.160605e-06,-5.139702e-06,...,1.998870e-06,2.659501e-06,3.480019e-06,3.214389e-06,3.496112e-06,3.580351e-06,1.858070e-06,2.468489e-06,2.096979e-06,0
19428,2.071741e-06,1.953985e-06,1.471937e-07,-1.463782e-06,-1.463443e-06,-3.596153e-06,-2.179754e-06,-1.122828e-06,-9.360783e-07,-1.914432e-06,...,-1.527506e-06,-1.949852e-06,-2.173926e-06,-2.548489e-06,-2.218234e-06,-3.495228e-06,-3.420214e-06,-3.451055e-06,-4.704019e-06,0
19429,7.384224e-06,7.708207e-06,9.351496e-06,9.013227e-06,1.125922e-05,1.204181e-05,1.140459e-05,8.744514e-06,8.424733e-06,7.520064e-06,...,1.167609e-06,4.388172e-07,8.032826e-07,5.066306e-07,4.639861e-07,4.091424e-07,-1.768839e-06,-1.995077e-06,-3.525502e-06,0
19430,-1.470589e-06,-2.534388e-06,-1.701899e-06,1.054022e-06,1.112566e-06,2.784270e-06,2.682054e-06,1.907659e-06,1.636055e-06,1.755764e-06,...,-1.251002e-07,1.325776e-07,-1.239752e-07,1.313204e-07,-1.226322e-07,1.299237e-07,-1.212041e-07,1.284802e-07,-1.197563e-07,0


In [18]:
babydf = babydf.dropna(how = 'any',axis=0)
len(babydf)

19374

In [19]:
babydf.to_csv('babydf16sec.csv')

In [20]:
babydf_copy=babydf

In [21]:
 babydf

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,annotation
0,-3.263788e-07,-1.235657e-06,-2.262840e-06,-7.633439e-08,-2.164782e-07,-9.267774e-07,-2.608645e-07,4.420569e-07,1.074870e-06,1.201442e-06,...,-2.108430e-08,-6.075327e-08,3.585114e-08,-7.522059e-08,-2.129476e-07,1.840603e-08,-4.448791e-08,-1.414767e-07,-1.127482e-07,0
1,-2.486152e-05,-2.503058e-05,-2.186363e-05,-2.480244e-05,-2.744258e-05,-3.197565e-05,-2.888203e-05,-2.740444e-05,-2.505538e-05,-2.380489e-05,...,-3.678916e-08,4.569411e-08,-1.483502e-09,-1.193529e-07,8.563138e-08,-8.082530e-08,6.143709e-08,7.245130e-08,-1.086324e-08,0
2,-2.500956e-07,3.655730e-07,1.434406e-06,1.338836e-06,1.395847e-06,1.193354e-06,4.941691e-07,-1.154113e-06,-1.678423e-06,-2.962719e-06,...,2.377878e-07,2.749303e-07,4.018399e-07,4.383263e-07,4.915250e-07,1.993675e-07,5.477542e-07,2.877784e-07,4.136224e-07,0
3,-1.232406e-05,-1.401220e-05,-1.413148e-05,-1.257527e-05,-1.504887e-05,-1.492065e-05,-1.218374e-05,-1.174109e-05,-1.214782e-05,-1.030836e-05,...,4.371654e-08,-2.403598e-08,-7.028860e-08,3.183884e-08,-7.033279e-09,-1.487082e-07,-2.021294e-08,1.308101e-07,-7.147434e-08,1
4,1.282229e-06,5.628105e-06,1.596520e-06,6.568708e-06,3.496735e-06,8.595807e-06,-1.840697e-06,4.598325e-05,8.422721e-05,6.828570e-05,...,-1.439245e-07,8.825039e-08,5.644418e-08,-7.229984e-08,-6.370705e-08,3.334085e-08,-1.182866e-07,-1.872063e-07,7.822432e-08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19427,-1.126685e-05,-1.047553e-05,-1.210227e-05,-1.163751e-05,-1.239892e-05,-1.125410e-05,-1.029162e-05,-9.223068e-06,-7.160605e-06,-5.139702e-06,...,1.998870e-06,2.659501e-06,3.480019e-06,3.214389e-06,3.496112e-06,3.580351e-06,1.858070e-06,2.468489e-06,2.096979e-06,0
19428,2.071741e-06,1.953985e-06,1.471937e-07,-1.463782e-06,-1.463443e-06,-3.596153e-06,-2.179754e-06,-1.122828e-06,-9.360783e-07,-1.914432e-06,...,-1.527506e-06,-1.949852e-06,-2.173926e-06,-2.548489e-06,-2.218234e-06,-3.495228e-06,-3.420214e-06,-3.451055e-06,-4.704019e-06,0
19429,7.384224e-06,7.708207e-06,9.351496e-06,9.013227e-06,1.125922e-05,1.204181e-05,1.140459e-05,8.744514e-06,8.424733e-06,7.520064e-06,...,1.167609e-06,4.388172e-07,8.032826e-07,5.066306e-07,4.639861e-07,4.091424e-07,-1.768839e-06,-1.995077e-06,-3.525502e-06,0
19430,-1.470589e-06,-2.534388e-06,-1.701899e-06,1.054022e-06,1.112566e-06,2.784270e-06,2.682054e-06,1.907659e-06,1.636055e-06,1.755764e-06,...,-1.251002e-07,1.325776e-07,-1.239752e-07,1.313204e-07,-1.226322e-07,1.299237e-07,-1.212041e-07,1.284802e-07,-1.197563e-07,0


In [22]:
from sklearn.metrics import precision_recall_fscore_support
y_true = [0, 1, 1, 0, 1, 1]
y_pred = [0, 1, 1, 0, 0, 1]
precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label=1)
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F-measure: {}'.format(fscore))
print('Support: {}'.format(support))

Precision: 1.0
Recall: 0.75
F-measure: 0.8571428571428571
Support: None


In [24]:
    t_recall_noNS, t_recall_NS = [], []
    t_precision_noNS, t_precision_NS = [], []


In [25]:
    for thresh in np.arange(0, 1, 0.01):
        precision, recall, fscore, support = \
                precision_recall_fscore_support(
                        y_true,
                        np.where(pred_proba[:,0] > thresh, 0, 1))
        recall_noNS, recall_NS = recall
        precision_noNS, precision_NS = precision

        t_recall_noNS.append(recall_noNS)
        t_recall_NS.append(recall_NS)

        t_precision_noNS.append(precision_noNS)
        t_precision_NS.append(precision_NS)


NameError: name 'y_test' is not defined